# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv("output_data/city.csv")
city_data_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,busselton,-33.6500,115.3333,61.93,71,100,4.27,AU,1666514489
1,1,kavaratti,10.5669,72.6420,81.91,72,24,1.10,IN,1666514491
2,2,ancud,-41.8697,-73.8203,44.10,81,75,10.42,CL,1666514491
3,3,bethel,41.3712,-73.4140,53.65,84,75,0.00,US,1666514202
4,4,barrow,71.2906,-156.7887,24.82,93,100,21.85,US,1666514494


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,port elizabeth,-33.9180,25.5701,76.10,73,0,8.05,ZA,1666514508
30,30,port said,31.2565,32.2841,73.38,78,0,6.91,EG,1666514519
42,42,faya,18.3851,42.4509,72.12,16,0,3.20,SA,1666514531
152,152,abha,18.2164,42.5053,71.85,21,0,5.75,SA,1666514572
241,241,cayenne,4.9333,-52.3333,75.24,94,0,0.00,GF,1666514611
391,391,boa vista,2.8197,-60.6733,76.98,88,0,3.44,BR,1666514873
411,411,anamur,36.0751,32.8369,74.37,26,0,4.07,TR,1666514817
429,429,mahon,39.8885,4.2658,77.56,83,0,8.05,ES,1666514906
536,536,jalu,29.0331,21.5482,72.09,42,0,9.15,LY,1666515003
570,570,vieux-habitants,16.0589,-61.7659,71.17,78,0,5.59,GP,1666515034


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
hotel_df=narrowed_city_df

In [7]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
16,16,port elizabeth,-33.9180,25.5701,76.10,73,0,8.05,ZA,1666514508,39 On Nile Guest House
30,30,port said,31.2565,32.2841,73.38,78,0,6.91,EG,1666514519,Aracan Hotel
42,42,faya,18.3851,42.4509,72.12,16,0,3.20,SA,1666514531,شيلة نسايم
152,152,abha,18.2164,42.5053,71.85,21,0,5.75,SA,1666514572,Abha Hotel
241,241,cayenne,4.9333,-52.3333,75.24,94,0,0.00,GF,1666514611,Hôtel Le Dronmi
391,391,boa vista,2.8197,-60.6733,76.98,88,0,3.44,BR,1666514873,Aipana Plaza Hotel
411,411,anamur,36.0751,32.8369,74.37,26,0,4.07,TR,1666514817,Luna Piena Beach Hotel
429,429,mahon,39.8885,4.2658,77.56,83,0,8.05,ES,1666514906,ARTIEM Capri
536,536,jalu,29.0331,21.5482,72.09,42,0,9.15,LY,1666515003,صاحبة الفخامة
570,570,vieux-habitants,16.0589,-61.7659,71.17,78,0,5.59,GP,1666515034,Habitation GETZ


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))